## Understanding Julia’s type system

### A single function in Julia may have multiple methods

You can define multiple methods for the same function with
different implementations, depending on the types of the passed arguments. You can
use the methods function to get the list of methods defined for a given function.

In [1]:
methods(cd)

# 4 methods for generic function "cd":
[1] cd() in Base.Filesystem at file.jl:94
[2] cd(dir::AbstractString) in Base.Filesystem at file.jl:89
[3] cd(f::Function) in Base.Filesystem at file.jl:147
[4] cd(f::Function, dir::AbstractString) in Base.Filesystem at file.jl:107

You can see that some of the arguments of the functions have type annotations; in this
case, they are ::Function and ::AbstractString, which restrict the types of val-
ues allowed by the given methods and change their behavior, depending on types of
passed values.

In [2]:
sum isa Function

true

In [3]:
typeof(sum)

typeof(sum) (singleton type of function sum, subtype of Function)

In [4]:
typeof(sum) == Function

false

we need to know that in Julia, types are orga-
nized in a hierarchy. This allows the bundling together of several types when defining
methods for functions. For instance, in the preceding example, the cd function can
take any function as an argument.

### Types in Julia are arranged in a hierarchy

In Julia, all types are arranged in a tree, and each type has a parent. This parent,
called a supertype, can be checked using the supertype function:

In [5]:
supertype(Function)

Any

In [6]:
supertype(UInt)

Unsigned

In [7]:
supertype(typeof(sum))

Function

In [8]:
typeof(sum)

typeof(sum) (singleton type of function sum, subtype of Function)

So, indeed, we see that the type of the sum function is a subtype of the Function type.

- The root type of the tree is called Any. All other types are subtypes of the Any
type. If you define a function without specifying its argument(s) type, as we did
in section 2.4, Julia assumes by default that the Any type is allowed; that is, you
can pass a value of any type to such a function.

- Only the types that are leaves can have instances (that is, have objects that are of
that specific type). The types that can be instantiated are called concrete. In other
words, if you have a value, you can be sure that its type is concrete and that it is
a leaf type. For this reason, there is no function whose type is Function. Every
function has its own unique concrete type that is a subtype of the Function type.

- The types that are not leaves of the type tree (for example, Any or Function)
cannot be instantiated. They serve only as intermediate types allowing for logi-
cal grouping of other types and are called abstract. You can find the list of sub-
types of an abstract type by calling the subtypes function.

In [9]:
supertypes(Int)

(Int64, Signed, Integer, Real, Number, Any)

In [10]:
supertypes(Function)

(Function, Any)

Only concrete types can be instantiated and cannot have concrete subtypes. You can
check whether a given type is concrete by using the isconcretetype function.
Abstract types cannot have instances but can have subtypes. You can check whether
a given type is abstract by using the isabstracttype function. Therefore, it is not
possible for a type to be both abstract and concrete.

However, some types are neither abstract nor concrete. You will encounter these
types in chapter 4 when you learn more about parametric types. An example of such
a type is Vector. (Note that this type has its parameter left out, and this is why it is
not concrete;

### Finding all supertypes of a type

In [11]:
function print_supertypes(T)
    println(T)
    T == Any || print_supertypes(supertype(T))
    return nothing
end


print_supertypes (generic function with 1 method)

In [12]:
print_supertypes(Int)

Int64
Signed
Integer
Real
Number
Any


As you can see, the type hierarchy is quite deep. This allows your functions to have
fine-grained control of the types of arguments they accept

It is a good practice to explicitly specify the value a function has to return  

If a function returns nothing, the Julia REPL
does not print any return value to the terminal. Therefore, in this exam-
ple, the only things that get printed are the types outputted by the
println(T) operation

### Finding all subtypes of a type

In [13]:
function print_subtypes(T, indent_level=0)
    println(" " ^ indent_level, T)
    for s in subtypes(T)
        print_subtypes(s, indent_level+4)
    end
    return nothing 
end

    

print_subtypes (generic function with 2 methods)

In [14]:
print_subtypes(Integer)

Integer
    Bool
    Signed
        BigInt
        Int128
        Int16
        Int32
        Int64
        Int8
    Unsigned
        UInt128
        UInt16
        UInt32
        UInt64
        UInt8


### Union of types

Referring to collections of types by using abstract types is useful. However, sometimes
you might want to specify a list of types that do not have the respective node (abstract
type) in the type tree. For instance, what if you want to allow only signed or unsigned
integers in your code but not Bool values? You could use the Union keyword.

In our
scenario, if you write Union{Signed, Unsigned}, you tell Julia that you allow any of
the types specified inside the curly brackets after the Union keyword.

In data science workflows, the Union keyword is often used when we specify a
union between a certain type and the Missing type. For instance, if you write
Union{String, Missing}, you indicate that a value must be a String but can
optionally be missing. Chapter 7 covers handling missing values in more detail.

### Deciding what type restrictions to put in method signature

In [20]:
print_supertypes(typeof([1., 2., 3]))

Vector{Float64}
DenseVector{Float64}
AbstractVector{Float64}
Any


In [24]:
print_supertypes(typeof(1:3))

UnitRange{Int64}
AbstractUnitRange{Int64}
OrdinalRange{Int64, Int64}
AbstractRange{Int64}
AbstractVector{Int64}
Any


In [25]:
AbstractVector

AbstractVector (alias for AbstractArray{T, 1} where T)

The typejoin function finds the narrowest type that is a supertype of types passed as
its arguments. You will not need this function often, but it is useful to confirm our
intuition in this case.

In [26]:
typejoin(typeof([1, 2]), typeof([1., 2.]))

Vector (alias for Array{T, 1} where T)